# Using pyspark Data-PreProcessing


In [ ]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [ ]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [ ]:
import pyspark.sql.functions as F
from pyspark.sql.types import *
import os

In [ ]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DoubleType, DateType

schema = StructType([
    StructField("properties", StructType([
        StructField("parameter", StructType([
            StructField("PS", MapType(StringType(),DoubleType()),True),
            StructField("PSC", MapType(StringType(),DoubleType()),True),
            StructField("T2MWET",  MapType(StringType(),DoubleType()),True),
            StructField("T2MDEW",  MapType(StringType(),DoubleType()),True),
            StructField("T2M",  MapType(StringType(),DoubleType()),True)
        ]), True)
    ]), True),
    StructField("geometry", StructType([
        StructField("coordinates", ArrayType(DoubleType()),True)
    ]), True)
])

city_df = spark.read.csv('file:///home/talentum/test-jupyter/Daily/program/Indian_Cities_Database.csv', header=True)

for city in city_df.collect():
    # print(city["City"])
    # Load the json file
    file_path = f'file:///home/talentum/test-jupyter/Hourly/dataset/{city["City"]}_*.json'

    data_df = spark.read.json(file_path, multiLine=True, schema=schema)
    # Load the json file
    #data_df = spark.read.json('file:///home/talentum/test-jupyter/Hourly/dataset/Adilabad_19.4000_78.3100_*.json', multiLine=True, schema=schema)

    #print(data_df.printSchema())
    #print(data_df.show())
    # print(city["City"])
    data_df = data_df.select(F.col("properties.parameter.*"),F.col("geometry.coordinates"))

    # create dataframe for features "PS","PSC","T2M","T2MWET","T2MDEW" 
    from pyspark.sql.functions import explode
    PS =  data_df.select('coordinates',explode(data_df.PS).alias("Date", "PS"))
    PSC = data_df.select(explode(data_df.PSC).alias("Date", "PSC"))
    T2MDEW = data_df.select(explode(data_df.T2MDEW).alias("Date", "T2MDEW"))
    T2MWET = data_df.select(explode(data_df.T2MWET).alias("Date", "T2MWET"))
    T2M = data_df.select(explode(data_df.T2M).alias("Date", "T2M"))



    from pyspark.sql.functions import desc
    temp_df = PS.join(PSC, PS.key == PSC.key, 'inner') \
    .join(T2M , PS.key == T2M.key, 'inner') \
    .join(T2MWET , PS.key == T2MWET.key, 'inner') \
    .join(T2MDEW , PS.key == T2MDEW.key, 'inner') \
    .select(PS.key, "coordinates","PS","PSC","T2M","T2MWET","T2MDEW")


    # another approach is by renaming.

    # final_df.show()
    # write dataframe to file in csv format
    temp_df.write.csv(f'file:///home/talentum/test-jupyter/Hourly/Output/{city["City"]}_temp', header=True)

In [ ]:
# for column in data_df.columns:
#     attribute =  data_df.select('coordinates',explode(data_df[column]).alias("Date", column))

#     # Perform a outer join between the empty DataFrame and the non-empty DataFrame on the "key" column
#     empty_df = empty_df.join(attribute, on="Date", how="outer")

# # Show the joined DataFrame
# #joined_df.show()
# empty_df.show()